## VI LAB 2 

## Data collection and preparation

For this project, the data was obtained directly from the NSF Award Search portal, which is the official source used by the National Science Foundation to publish information about funded grants (referred to administratively as “awards”). This source was chosen because it provides all the attributes required by the project specification, including state, directorate, award dates, and awarded amounts, and because it allows filtering by both time period and award status.

Two datasets were collected to fully satisfy the project requirements. The first dataset contains all NSF grants awarded during the last five years (2020–2024) and serves as the baseline for analyzing current funding distribution and evolution. The second dataset contains NSF grants that were explicitly terminated during the Trump administration (2017–2021), filtered using the “terminated” award status. This separation is intentional and necessary, as the project explicitly requires analyzing both recent grants and historical cancellations from a different political period.

Due to export limitations of the NSF portal, the 2020–2024 dataset was downloaded in multiple smaller time ranges and later merged. This approach ensured complete coverage while preserving data integrity and consistency.

## Data cleaning

All major data cleaning was performed in OpenRefine to keep the Python notebook focused on visualization rather than preprocessing. In OpenRefine, column names were standardized across datasets, unnecessary administrative fields were removed, and monetary values were converted to numeric format. A derived year attribute was created from the award start date to support temporal analysis. Additionally, a categorical flag (cancelled_trump) was introduced to clearly distinguish between baseline grants and Trump-era terminated grants.

After cleaning, the datasets were exported as clean CSV files and loaded into the Python notebook. Only minimal preprocessing was performed in Python, consisting of type checks, column name normalization, and the creation of aggregated DataFrames for each visualization task.

In [ ]:
import pandas as pd
import altair as alt

# Performance: required by the project (datasets > 5000 rows)
alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

In [2]:
# load datasets

base_path = "."

df_grants = pd.read_csv(
    f"{base_path}/NSF_Grants_Last5Years_Clean.csv"
)

df_trump = pd.read_csv(
    f"{base_path}/trump17-21-csv.csv"
)


In [3]:
# Ensure correct dtypes
df_grants["year"] = df_grants["year"].astype(int)
df_grants["award_amount"] = pd.to_numeric(df_grants["award_amount"], errors="coerce")

df_trump["year"] = df_trump["year"].astype(int)
df_trump["award_amount"] = pd.to_numeric(df_trump["award_amount"], errors="coerce")


In [4]:
# Drop rows with critical missing values
df_grants = df_grants.dropna(subset=["state", "directorate", "year"])
df_trump = df_trump.dropna(subset=["directorate"])


In [5]:
year_selection = alt.selection_point(
    fields=["year"],
    bind=alt.binding_select(
        options=sorted(df_grants["year"].unique()),
        name="Year: "
    ),
    value=sorted(df_grants["year"].unique())[0]
)

state_selection = alt.selection_point(
    fields=["state"],
    bind=alt.binding_select(
        options=sorted(df_grants["state"].unique()),
        name="State: "
    )
)


## Q1: How are the grants distributed by states every year?

In [19]:
import altair as alt
import pandas as pd

# 0. ENSURE RENDERER IS CORRECT (Fixes the error)
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
# Re-ensure q1_df is ready
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. CREATE SELECTIONS
# Define the missing year_selection
year_selection = alt.selection_point(
    fields=["year"],
    bind=alt.binding_select(
        options=sorted(q1_df["year"].unique()), name="Select Year: "
    ),
    value=[{"year": 2021}],  # Set a default value to avoid empty chart initially
)

state_click = alt.selection_point(fields=["state"], empty="all")

# 3. CREATE CHARTS
q1_bars = (
    alt.Chart(q1_df)
    .mark_bar()
    .encode(
        x=alt.X("state:N", sort="-y", title="State"),
        y=alt.Y("grants_count:Q", title="Number of grants"),
        color=alt.condition(
            state_click,
            alt.Color(
                "grants_count:Q", scale=alt.Scale(scheme="blues"), title="Grants count"
            ),
            alt.value("lightgray"),
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total amount ($)", format=",.0f"),
        ],
    )
    .add_params(year_selection, state_click)
    .transform_filter(year_selection)
    .properties(width=750, height=380, title="Q1 — Grants by State")
)

q1_state_trend = (
    alt.Chart(q1_df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("grants_count:Q", title="Grants"),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total amount ($)", format=",.0f"),
        ],
    )
    .transform_filter(state_click)
    .properties(width=750, height=180, title="Selected State — Grants over Time")
)

# 4. DISPLAY
(q1_bars & q1_state_trend)

alt.VConcatChart(...)

In [26]:
import altair as alt
import pandas as pd
from vega_datasets import data

# 0. FIX THE RENDERER (Crucial for avoiding errors)
alt.data_transformers.enable("default")

# 1. PREPARE THE DATA
# We ensure the aggregation is correct
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. DEFINE THE MAP DATA
# We need to map State Abbreviations (AK, AL) to FIPS Codes (02, 01) for the map to work.
state_to_fips = {
    "WA": "53",
    "DE": "10",
    "DC": "11",
    "WI": "55",
    "WV": "54",
    "HI": "15",
    "FL": "12",
    "WY": "56",
    "PR": "72",
    "NJ": "34",
    "NM": "35",
    "TX": "48",
    "LA": "22",
    "NC": "37",
    "ND": "38",
    "NE": "31",
    "TN": "47",
    "NY": "36",
    "PA": "42",
    "AK": "02",
    "NV": "32",
    "NH": "33",
    "VA": "51",
    "CO": "08",
    "CA": "06",
    "AL": "01",
    "AR": "05",
    "VT": "50",
    "IL": "17",
    "GA": "13",
    "IN": "18",
    "IA": "19",
    "MA": "25",
    "AZ": "04",
    "ID": "16",
    "CT": "09",
    "ME": "23",
    "MD": "24",
    "OK": "40",
    "OH": "39",
    "UT": "49",
    "MO": "29",
    "MN": "27",
    "MI": "26",
    "RI": "44",
    "KS": "20",
    "MT": "30",
    "MS": "28",
    "SC": "45",
    "KY": "21",
    "OR": "41",
    "SD": "46",
}

# Create a lookup dataframe
fips_df = pd.DataFrame(list(state_to_fips.items()), columns=["state", "id"])
fips_df["id"] = fips_df["id"].astype(int)  # Ensure ID is integer to match topojson

# 3. AGGREGATE DATA BY STATE FIRST (for the map - sum across all years)
# This gives us one row per state with total funding
q1_map_agg = (
    q1_df.groupby("state")
    .agg(total_amount=("total_amount", "sum"))
    .reset_index()
)

# Merge FIPS IDs into aggregated data
q1_map_data_agg = q1_map_agg.merge(fips_df, on="state", how="inner")

# Keep the full data with FIPS for the trend chart
q1_map_data_full = q1_df.merge(fips_df, on="state", how="inner")

# 4. CREATE THE INTERACTION (Compatible with Altair 4 & 5)
# "Clicking a state selects it"
try:
    # Try new syntax first
    state_select = alt.selection_point(fields=["id"], empty="all")
except AttributeError:
    # Fallback for older Altair versions
    state_select = alt.selection_single(fields=["id"], empty="all")

# 5. DRAW THE MAP (Overview)
# We use a standard US map topology
us_states = alt.topo_feature(data.us_10m.url, "states")

map_chart = (
    alt.Chart(us_states)
    .mark_geoshape(stroke="white", strokeWidth=0.5)
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(q1_map_data_agg, "id", ["total_amount", "state"]),
        default=0  # Default value for states without data
    )
    .encode(
        color=alt.Color(
            "total_amount:Q", 
            scale=alt.Scale(scheme="blues"), 
            title="Total Funding ($)",
            legend=alt.Legend(format="$,.0f")
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("total_amount:Q", title="Total Funding", format="$,.0f")
        ],
    )
    .add_params(state_select)
    .project(type="albersUsa")
    .properties(
        width=700, height=400, title="Q1: Funding by State (Click to Filter Trend)"
    )
)

# 6. DRAW THE TREND LINE (Detail)
# Shows the trend for the SELECTED state
# Filter the trend data based on the selected state's id
trend_chart = (
    alt.Chart(q1_map_data_full)
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("total_amount:Q", title="Total Amount ($)", axis=alt.Axis(format="$,.0f")),
        color=alt.value("lightblue"),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("total_amount:Q", title="Total Amount", format="$,.0f")
        ],
    )
    .transform_filter(state_select)  # This will filter by the id field in the selection
    .add_params(state_select)
    .properties(width=700, height=200, title="Funding Trend for Selected State")
)

# 7. COMBINE
final_viz = map_chart & trend_chart
final_viz

alt.VConcatChart(...)

In [33]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. CREATE SELECTIONS
min_year = int(q1_df["year"].min())
max_year = int(q1_df["year"].max())

slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    # Modern Altair
    year_select = alt.selection_point(
        name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}]
    )
    state_select = alt.selection_point(
        name="state_select", fields=["state"], empty="all"
    )
except AttributeError:
    # Older Altair
    year_select = alt.selection_single(
        name="year_select", fields=["year"], bind=slider, init={"year": max_year}
    )
    state_select = alt.selection_single(
        name="state_select", fields=["state"], empty="all"
    )

# 3. CHART A: MAIN BAR CHART
bars = (
    alt.Chart(q1_df)
    .mark_bar()
    .encode(
        x=alt.X("state:N", sort="-y", title="State"),
        y=alt.Y("grants_count:Q", title="Number of Grants"),
        color=alt.condition(
            state_select,
            alt.Color("grants_count:Q", scale=alt.Scale(scheme="blues"), legend=None),
            alt.value("#f0f0f0"),  # Very light gray for unselected
        ),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("grants_count:Q"),
            alt.Tooltip("total_amount:Q", format="$,.0f"),
        ],
    )
    .transform_filter(year_select)
    .properties(width=550, height=400, title="Grants Distribution")
)

# 4. CHART B: KPI TEXT (Subtle & Professional)
# We layer two text marks: one for the label, one for the number
base_text = (
    alt.Chart(q1_df).transform_filter(year_select).transform_filter(state_select)
)

# Layer 1: The Label "Total Funding"
label = base_text.mark_text(
    align="center", color="#888", fontSize=14, dy=-15  # Light gray  # Move up slightly
).encode(text=alt.value("Total Funding"), y=alt.value(200), x=alt.value(100))

# Layer 2: The Value (The Number)
value = base_text.mark_text(
    align="center",
    color="#444",  # Darker gray (but not black)
    fontSize=24,  # Smaller than 40
    fontWeight="bold",
    dy=15,  # Move down slightly
).encode(
    text=alt.Text("sum(total_amount):Q", format="$,.0f"),
    y=alt.value(200),
    x=alt.value(100),
)

kpi_section = (label + value).properties(width=200, height=400)

# 5. DISPLAY (Combine and Attach Slider)
# Attaching add_params to the final object puts the slider at the bottom
final_q1 = (
    (bars | kpi_section)
    .add_params(year_select, state_select)
    .resolve_scale(color="independent")
)

final_q1

alt.HConcatChart(...)

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
# Aggregate data by State and Year
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(
        grants_count=("award_id", "count"),
        total_amount=("award_amount", "sum")
    )
    .reset_index()
)

# 2. MAP DATA PREPARATION (FIPS CODES)
state_to_fips = {
    "WA": "53", "DE": "10", "DC": "11", "WI": "55", "WV": "54", "HI": "15",
    "FL": "12", "WY": "56", "PR": "72", "NJ": "34", "NM": "35", "TX": "48",
    "LA": "22", "NC": "37", "ND": "38", "NE": "31", "TN": "47", "NY": "36",
    "PA": "42", "AK": "02", "NV": "32", "NH": "33", "VA": "51", "CO": "08",
    "CA": "06", "AL": "01", "AR": "05", "VT": "50", "IL": "17", "GA": "13",
    "IN": "18", "IA": "19", "MA": "25", "AZ": "04", "ID": "16", "CT": "09",
    "ME": "23", "MD": "24", "OK": "40", "OH": "39", "UT": "49", "MO": "29",
    "MN": "27", "MI": "26", "RI": "44", "KS": "20", "MT": "30", "MS": "28",
    "SC": "45", "KY": "21", "OR": "41", "SD": "46",
}
fips_df = pd.DataFrame(list(state_to_fips.items()), columns=["state", "id"])
fips_df["id"] = fips_df["id"].astype(int)

# Merge FIPS into main data for the Map
q1_full = q1_df.merge(fips_df, on="state", how="inner")

# 3. INTERACTION SETUP
# A. Year Slider
min_year = int(q1_full["year"].min())
max_year = int(q1_full["year"].max())
slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    year_select = alt.selection_point(name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}])
    state_select = alt.selection_point(name="state_select", fields=["id"], empty="all") 
except AttributeError:
    # Fallback for older Altair versions
    year_select = alt.selection_single(name="year_select", fields=["year"], bind=slider, init={"year": max_year})
    state_select = alt.selection_single(name="state_select", fields=["id"], empty="all")

# 4. CHART A: THE MAP (Overview)
us_states = alt.topo_feature(data.us_10m.url, "states")

# Base layer (Gray background)
map_base = (
    alt.Chart(us_states)
    .mark_geoshape(fill="lightgray", stroke="white")
    .project(type="albersUsa")
)

# Data layer (Colored by Funding)
map_layer = (
    alt.Chart(us_states)
    .mark_geoshape(stroke="white")
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(q1_full, "id", ["total_amount", "state", "year"])
    )
    .transform_filter(year_select) # Filter Map by Year Slider
    .encode(
        color=alt.Color(
            "total_amount:Q", 
            scale=alt.Scale(scheme="blues"), 
            title="Funding ($)"
        ),
        tooltip=["state:N", alt.Tooltip("total_amount:Q", format="$,.0f")]
    )
    .project(type="albersUsa")
)

the_map = (map_base + map_layer).properties(
    width=500, height=350, 
    title="Geographic Distribution (Select Year)"
)

# 5. CHART B: THE BAR CHART (Ranking)
the_bars = (
    alt.Chart(q1_full)
    .mark_bar()
    .encode(
        x=alt.X("total_amount:Q", title="Total Funding ($)"),
        y=alt.Y("state:N", sort="-x", title="State"),
        color=alt.condition(
            state_select, 
            alt.Color("total_amount:Q", scale=alt.Scale(scheme="blues"), legend=None),
            alt.value("lightgray")
        ),
        tooltip=["state", "total_amount"]
    )
    .transform_filter(year_select)
    .properties(
        width=200, height=350, 
        title="Ranked by Funding"
    )
)

# 6. CHART C: THE EVOLUTION (Trend Line)
the_trend = (
    alt.Chart(q1_full)
    .mark_line(point=True, stroke="#4c78a8")
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("total_amount:Q", title="Total Funding ($)"),
        tooltip=["state", "year", alt.Tooltip("total_amount", format="$,.0f")]
    )
    .transform_filter(state_select) # Only show trend for clicked state
    .properties(
        width=750, height=200,
        title="Evolution of Funding (Selected State)"
    )
)

# 7. FINAL DASHBOARD ASSEMBLY
# We attach the parameters here to ensure they are available globally
final_dashboard = ((the_map | the_bars) & the_trend).add_params(
    year_select, 
    state_select
).resolve_scale(
    color='independent'
)

final_dashboard

alt.VConcatChart(...)

In [37]:
import altair as alt
import pandas as pd
from vega_datasets import data

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. MAP DATA PREPARATION
state_to_fips = {
    "WA": "53",
    "DE": "10",
    "DC": "11",
    "WI": "55",
    "WV": "54",
    "HI": "15",
    "FL": "12",
    "WY": "56",
    "PR": "72",
    "NJ": "34",
    "NM": "35",
    "TX": "48",
    "LA": "22",
    "NC": "37",
    "ND": "38",
    "NE": "31",
    "TN": "47",
    "NY": "36",
    "PA": "42",
    "AK": "02",
    "NV": "32",
    "NH": "33",
    "VA": "51",
    "CO": "08",
    "CA": "06",
    "AL": "01",
    "AR": "05",
    "VT": "50",
    "IL": "17",
    "GA": "13",
    "IN": "18",
    "IA": "19",
    "MA": "25",
    "AZ": "04",
    "ID": "16",
    "CT": "09",
    "ME": "23",
    "MD": "24",
    "OK": "40",
    "OH": "39",
    "UT": "49",
    "MO": "29",
    "MN": "27",
    "MI": "26",
    "RI": "44",
    "KS": "20",
    "MT": "30",
    "MS": "28",
    "SC": "45",
    "KY": "21",
    "OR": "41",
    "SD": "46",
}
fips_df = pd.DataFrame(list(state_to_fips.items()), columns=["state", "id"])
fips_df["id"] = fips_df["id"].astype(int)
q1_full = q1_df.merge(fips_df, on="state", how="inner")

# 3. INTERACTION SETUP
min_year = int(q1_full["year"].min())
max_year = int(q1_full["year"].max())
slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    year_select = alt.selection_point(
        name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}]
    )
    state_select = alt.selection_point(name="state_select", fields=["id"], empty="all")
except AttributeError:
    year_select = alt.selection_single(
        name="year_select", fields=["year"], bind=slider, init={"year": max_year}
    )
    state_select = alt.selection_single(name="state_select", fields=["id"], empty="all")

# 4. CHART A: THE MAP (Smaller & Left)
us_states = alt.topo_feature(data.us_10m.url, "states")

map_base = (
    alt.Chart(us_states)
    .mark_geoshape(fill="lightgray", stroke="white")
    .project(type="albersUsa")
)

map_layer = (
    alt.Chart(us_states)
    .mark_geoshape(stroke="white")
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(q1_full, "id", ["total_amount", "state", "year"]),
    )
    .transform_filter(year_select)
    .encode(
        color=alt.Color(
            "total_amount:Q", scale=alt.Scale(scheme="blues"), title="Funding ($)"
        ),
        tooltip=["state:N", alt.Tooltip("total_amount:Q", format="$,.0f")],
    )
    .project(type="albersUsa")
)

the_map = (map_base + map_layer).properties(
    width=300, height=350, title="Geographic Overview"  # REDUCED WIDTH (was 500)
)

# 5. CHART B: THE BAR CHART (Larger & Right)
the_bars = (
    alt.Chart(q1_full)
    .mark_bar()
    .encode(
        x=alt.X("total_amount:Q", title="Total Funding ($)"),
        y=alt.Y("state:N", sort="-x", title="State"),
        color=alt.condition(
            state_select,
            alt.Color("total_amount:Q", scale=alt.Scale(scheme="blues"), legend=None),
            alt.value("lightgray"),
        ),
        tooltip=["state", "total_amount"],
    )
    .transform_filter(year_select)
    .properties(
        width=400,  # INCREASED WIDTH (was 200)
        height=350,
        title="Ranked Funding by State",
    )
)

# 6. CHART C: THE EVOLUTION (Uniform Color)
the_trend = (
    alt.Chart(q1_full)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("total_amount:Q", title="Total Funding ($)"),
        color=alt.value("steelblue"),  # UNIFORM COLOR (Fixed Blue)
        tooltip=["state", "year", alt.Tooltip("total_amount", format="$,.0f")],
    )
    .transform_filter(state_select)
    .properties(
        width=750,  # Matches the sum of top charts approx (300+400 + padding)
        height=200,
        title="Evolution of Funding (Selected State)",
    )
)

# 7. FINAL DASHBOARD
final_dashboard = (
    ((the_bars | the_map) & the_trend)
    .add_params(year_select, state_select)
    .resolve_scale(color="independent")
)

final_dashboard

alt.VConcatChart(...)

In [69]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
# We need detailed data for the trend (state + year), which you already have in q1_df
q1_df = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. CREATE SELECTIONS
min_year = int(q1_df["year"].min())
max_year = int(q1_df["year"].max())

slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    # Modern Altair
    year_select = alt.selection_point(
        name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}]
    )
    state_select = alt.selection_point(
        name="state_select", fields=["state"], empty="all"
    )
except AttributeError:
    # Older Altair
    year_select = alt.selection_single(
        name="year_select", fields=["year"], bind=slider, init={"year": max_year}
    )
    state_select = alt.selection_single(
        name="state_select", fields=["state"], empty="all"
    )

# 3. LEFT COLUMN: MAIN BAR CHART
bars = (
    alt.Chart(q1_df)
    .mark_bar()
    .encode(
        x=alt.X("state:N", sort="-y", title="State"),
        y=alt.Y("grants_count:Q", title="Number of Grants"),
        color=alt.condition(
            state_select,
            alt.Color("grants_count:Q", scale=alt.Scale(scheme="blues"), legend=None),
            alt.value("#f0f0f0"),  # Light gray for unselected
        ),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("grants_count:Q"),
            alt.Tooltip("total_amount:Q", format="$,.0f"),
        ],
    )
    .transform_filter(year_select)
    .properties(width=500, height=400, title="Grants Distribution (Click a bar)")
)

# 4. RIGHT COLUMN: EVOLUTION + KPI

# A. Evolution Chart (Top Right)
# Shows the trend for the selected state over ALL years
trend_chart = (
    alt.Chart(q1_df)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X("year:O", title="Year"),  # Ordinal to show specific years
        y=alt.Y("total_amount:Q", title="Total Amount ($)", axis=alt.Axis(format="~s")),
        color=alt.value("#4c78a8"),  # Blue to match the bar chart color scheme
        tooltip=[alt.Tooltip("year:O"), alt.Tooltip("total_amount:Q", format="$,.0f")],
    )
    .transform_filter(state_select)  # This is the magic: Filter by the click!
    .properties(
        width=250,
        height=180,  # Half the height of the main chart roughly
        title="History (Selected State)",
    )
)

# B. KPI Text (Bottom Right)
base_text = (
    alt.Chart(q1_df).transform_filter(year_select).transform_filter(state_select)
)

# Layer 1: Label
label = base_text.mark_text(align="center", color="#888", fontSize=14, dy=-15).encode(
    text=alt.value("Total Funding (Selected Year)"), y=alt.value(75), x=alt.value(110)
)

# Layer 2: Value
value = base_text.mark_text(
    align="center",
    color="#444",
    fontSize=24,
    fontWeight="bold",
    dy=15,
).encode(
    text=alt.Text("sum(total_amount):Q", format="$,.0f"),
    y=alt.value(75),  # Centered vertically in its box
    x=alt.value(110),  # Centered horizontally
)

kpi_section = (label + value).properties(width=225, height=155)

# 5. ASSEMBLE
# Right column is Trend on top of KPI
right_col = trend_chart & kpi_section

# Final is Bars on left of Right Column
final_q1 = (
    (bars | right_col)
    .add_params(year_select, state_select)
    .resolve_scale(color="independent")
)

final_q1

alt.HConcatChart(...)

To analyze grant distribution by state, I implemented a **composite dashboard** centered on a sorted bar chart. A bar chart was chosen over a choropleth map for the primary view because it allows for precise ranking and direct comparison of grant magnitudes, which are often obscured by geography in map views.

The design follows Shneiderman’s mantra: the bars provide the **overview** for the selected year. The **filtering** mechanism (year slider) enables temporal exploration, allowing users to observe shifts in distribution over time. **Details-on-demand** are achieved through linking: clicking a specific state isolates it visually (using a "focus+context" gray/blue color scheme) and triggers the side panels.

The right-hand column adds critical context: the **trend line** reveals the selected state's 5-year funding trajectory (evolution), while the **KPI text** provides the precise financial figure ($), bridging the gap between abstract patterns and exact data. This structure answers the question by showing both the relative standing of states and their individual historical performance.

## Q2: How are the grants distributed per directorates? And for a certain year?

In [70]:
# Q2 aggregation: grants per directorate per year
q2_df = (
    df_grants
    .groupby(["directorate", "year"])
    .agg(
        grants_count=("award_id", "count"),
        total_amount=("award_amount", "sum")
    )
    .reset_index()
)

q2_df.head()


,directorate,year,grants_count,total_amount
0,AGS,2020,16,3300718.0
1,AGS,2022,16,1375227.0
2,AGS,2023,6,871122.0
3,AGS,2024,7,1192193.0
4,AST,2020,9,2094148.0


In [ ]:
dir_click = alt.selection_point(fields=["directorate"], empty="all")

In [41]:
q2_overview = (
    alt.Chart(q2_df)
    .mark_bar()
    .encode(
        y=alt.Y("directorate:N", sort="-x", title="Directorate"),
        x=alt.X("grants_count:Q", title="Number of grants"),
        color=alt.condition(
            dir_click,
            alt.Color("grants_count:Q", scale=alt.Scale(scheme="blues"), title="Grants count"),
            alt.value("lightgray")
        ),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total amount ($)", format=",.0f"),
        ],
    )
    .add_params(year_selection, dir_click)
    .transform_filter(year_selection)
    .properties(
        title="Q2 — Grants by Directorate (select a year + click a directorate)",
        width=750,
        height=420,
    )
)


In [42]:
q2_trend = (
    alt.Chart(q2_df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("grants_count:Q", title="Number of grants"),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total amount ($)", format=",.0f"),
        ],
    )
    .transform_filter(dir_click)
    .properties(
        title="Selected directorate — grants over time",
        width=750,
        height=180,
    )
)


In [43]:
(q2_overview & q2_trend)


alt.VConcatChart(...)

In [72]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
q2_df = (
    df_grants.groupby(["directorate", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. CREATE SELECTIONS
min_year = int(q2_df["year"].min())
max_year = int(q2_df["year"].max())

slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    # Modern Altair
    year_select = alt.selection_point(
        name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}]
    )
    dir_select = alt.selection_point(
        name="dir_select", fields=["directorate"], empty="all"
    )
except AttributeError:
    # Older Altair
    year_select = alt.selection_single(
        name="year_select", fields=["year"], bind=slider, init={"year": max_year}
    )
    dir_select = alt.selection_single(
        name="dir_select", fields=["directorate"], empty="all"
    )

# 3. LEFT COLUMN: HORIZONTAL BARS (The Overview)
bars = (
    alt.Chart(q2_df)
    .mark_bar()
    .encode(
        x=alt.X("grants_count:Q", title="Number of Grants"),
        y=alt.Y("directorate:N", sort="-x", title="Directorate"),
        color=alt.condition(
            dir_select,
            alt.Color("grants_count:Q", scale=alt.Scale(scheme="teals"), legend=None),
            alt.value("#f0f0f0"),
        ),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total Amount ($)", format=",.0f"),
        ],
    )
    .transform_filter(year_select)
    .properties(width=450, height=550, title="Grants by Directorate (Click to Filter)")
)

# 4. RIGHT COLUMN: TREND + KPI

# A. Trend Line (The History) - REMOVED 'PADDING' TO FIX ERROR
trend_chart = (
    alt.Chart(q2_df)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y(
            "total_amount:Q", title="Total Funding ($)", axis=alt.Axis(format="~s")
        ),
        color=alt.value("teal"),
        tooltip=[
            alt.Tooltip("directorate:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("total_amount:Q", format="$,.0f"),
        ],
    )
    .transform_filter(dir_select)
    .properties(
        width=300,
        height=200,
        title="Funding History (Selected Directorate)",
        # padding removed here
    )
)

# B. KPI Text (The Detail)
base_text = alt.Chart(q2_df).transform_filter(year_select).transform_filter(dir_select)

label = base_text.mark_text(align="center", color="#888", fontSize=14, dy=-15).encode(
    text=alt.value("Total Funding (Selected Year)"), y=alt.value(75), x=alt.value(150)
)

value = base_text.mark_text(
    align="center", color="#444", fontSize=24, fontWeight="bold", dy=15
).encode(
    text=alt.Text("sum(total_amount):Q", format="$,.0f"),
    y=alt.value(75),
    x=alt.value(150),
)

kpi_section = (label + value).properties(width=300, height=155)

# 5. ASSEMBLE
# We vertically stack Trend + KPI
right_col = trend_chart & kpi_section

# We horizontally stack Bars | Right Column
final_q2 = (
    (bars | right_col)
    .add_params(year_select, dir_select)
    .resolve_scale(color="independent")
)

final_q2

alt.HConcatChart(...)

In [ ]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
q2_df = (
    df_grants.groupby(["directorate", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

# 2. CREATE SELECTIONS
min_year = int(q2_df["year"].min())
max_year = int(q2_df["year"].max())

slider = alt.binding_range(min=min_year, max=max_year, step=1, name="Select Year: ")

try:
    year_select = alt.selection_point(
        name="year_select", fields=["year"], bind=slider, value=[{"year": max_year}]
    )
    dir_select = alt.selection_point(
        name="dir_select", fields=["directorate"], empty="all"
    )
except AttributeError:
    year_select = alt.selection_single(
        name="year_select", fields=["year"], bind=slider, init={"year": max_year}
    )
    dir_select = alt.selection_single(
        name="dir_select", fields=["directorate"], empty="all"
    )

# 3. LEFT COLUMN: HORIZONTAL BARS
bars = (
    alt.Chart(q2_df)
    .mark_bar()
    .encode(
        x=alt.X("grants_count:Q", title="Number of Grants"),
        y=alt.Y("directorate:N", sort="-x", title="Directorate"),
        color=alt.condition(
            dir_select,
            alt.Color(
                "grants_count:Q",
                scale=alt.Scale(scheme="blues"),
                legend=None,
            ),
            alt.value("#f0f0f0"),
        ),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("grants_count:Q", title="Grants"),
            alt.Tooltip("total_amount:Q", title="Total Amount ($)", format=",.0f"),
        ],
    )
    .transform_filter(year_select)
    .properties(width=450, height=550, title="Grants by Directorate (Click to Filter)")
)

# 4. RIGHT COLUMN COMPONENTS

# A. Spacer para crear gap superior
top_spacer = (
    alt.Chart(q2_df)
    .mark_rect()
    .encode(opacity=alt.value(0))
    .properties(width=300, height=30)
)

# B. Trend Line
trend_chart = (
    alt.Chart(q2_df)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y(
            "total_amount:Q", title="Total Funding ($)", axis=alt.Axis(format="~s")
        ),
        color=alt.value("#4c78a8"),
        tooltip=[
            alt.Tooltip("directorate:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("total_amount:Q", format="$,.0f"),
        ],
    )
    .transform_filter(dir_select)
    .properties(
        width=300,
        height=220,
        title="Funding History (Selected Directorate)",
    )
)

# C. KPI Text
base_text = alt.Chart(q2_df).transform_filter(year_select).transform_filter(dir_select)

label = base_text.mark_text(align="center", color="#888", fontSize=14, dy=-15).encode(
    text=alt.value("Total Funding (Selected Year)"), y=alt.value(75), x=alt.value(100)
)

value = base_text.mark_text(
    align="center", color="#444", fontSize=24, fontWeight="bold", dy=15
).encode(
    text=alt.Text("sum(total_amount):Q", format="$,.0f"),
    y=alt.value(75),
    x=alt.value(100),
)

kpi_section = (label + value).properties(width=300, height=100)

# D. Legend
legend_chart = (
    alt.Chart(q2_df)
    .mark_circle(opacity=0)
    .encode(
        color=alt.Color(
            "grants_count:Q",
            scale=alt.Scale(scheme="blues"),
            legend=alt.Legend(
                title="Grant Count Intensity",
                orient="bottom",
                direction="horizontal",
                titleAnchor="middle",
                gradientLength=200,
            ),
        )
    )
    .transform_filter(year_select)
    .properties(width=300, height=40)
)

# 5. ASSEMBLE
# Ahora incluimos el spacer al inicio
right_col = alt.vconcat(
    top_spacer,
    trend_chart,
    kpi_section,
    legend_chart,
    spacing=5,
)

# Final Assembly
final_q2 = (
    (bars | right_col)
    .add_params(year_select, dir_select)
    .resolve_scale(color="independent")
    .configure_view(stroke=None)
    .configure_concat(spacing=10)
)

final_q2

alt.HConcatChart(...)

comments


## Q3: Are the cancelled grants especially hitting a certain directorate?

In [98]:
import pandas as pd
import altair as alt

# --- Aggregations ---
q3_cancel_df = (
    df_trump.groupby(["directorate"])
    .agg(
        cancelled_count=("award_id", "count"), cancelled_amount=("award_amount", "sum")
    )
    .reset_index()
)

q3_base_df = (
    df_grants.groupby(["directorate"])
    .agg(base_count=("award_id", "count"), base_amount=("award_amount", "sum"))
    .reset_index()
)

q3_df = q3_base_df.merge(q3_cancel_df, on="directorate", how="outer").fillna(0)

q3_df["cancel_rate"] = (
    q3_df["cancelled_count"] / q3_df["base_count"].replace(0, pd.NA)
).fillna(0)

# ✅ IMPORTANT: for the scatter (especially if you use log), remove base_count == 0
q3_scatter_df = q3_df[(q3_df["base_count"] > 0) & (q3_df["cancelled_count"] > 0)].copy()

# --- Selection ---
dir_sel = alt.selection_point(fields=["directorate"], empty="all")

# --- Scatter (overview) ---
# Option A (recommended): keep it LINEAR to match your expected "spread"
q3_scatter = (
    alt.Chart(q3_scatter_df)
    .mark_circle(opacity=0.8, stroke="black", strokeWidth=0.4)
    .encode(
        x=alt.X("base_count:Q", title="Baseline grants (last 5 years)"),
        y=alt.Y("cancelled_count:Q", title="Cancelled grants (Trump era)"),
        size=alt.Size("cancelled_amount:Q", title="Cancelled amount ($)", legend=None),
        color=alt.Color(
            "cancel_rate:Q",
            title="Cancellation rate",
            scale=alt.Scale(scheme="oranges"),
        ),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("base_count:Q", title="Baseline grants"),
            alt.Tooltip("cancelled_count:Q", title="Cancelled grants"),
            alt.Tooltip("cancel_rate:Q", title="Cancel rate", format=".2%"),
            alt.Tooltip(
                "cancelled_amount:Q", title="Cancelled amount ($)", format=",.0f"
            ),
        ],
    )
    .add_params(dir_sel)
    .properties(
        width=750,
        height=380,
        title="Q3 — Cancelled grants vs baseline distribution (by directorate)",
    )
)

# --- Bars ---
n_dirs = q3_df[q3_df["cancelled_count"] > 0]["directorate"].nunique()
rank_height = max(300, n_dirs * 18)

q3_bars = (
    alt.Chart(q3_df)
    .mark_bar()
    .encode(
        y=alt.Y(
            "directorate:N",
            sort="-x",
            title="Directorate",
            axis=alt.Axis(labelLimit=200),
        ),
        x=alt.X("cancelled_count:Q", title="Cancelled grants"),
        color=alt.condition(dir_sel, alt.value("#d95f02"), alt.value("lightgray")),
        tooltip=[
            alt.Tooltip("directorate:N"),
            alt.Tooltip("cancelled_count:Q", title="Cancelled grants"),
            alt.Tooltip("cancel_rate:Q", title="Cancel rate", format=".2%"),
        ],
    )
    .transform_filter(alt.datum.cancelled_count > 0)
    .add_params(dir_sel)
    .properties(
        width=750, height=rank_height, title="Cancelled grants ranking (click to focus)"
    )
)

# --- Trend (details on demand) ---
q3_cancel_by_year = (
    df_trump.groupby(["directorate", "year"])
    .agg(
        cancelled_count=("award_id", "count"), cancelled_amount=("award_amount", "sum")
    )
    .reset_index()
)

q3_trend = (
    alt.Chart(q3_cancel_by_year)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year (Trump era)"),
        y=alt.Y("cancelled_count:Q", title="Cancelled grants"),
        tooltip=[
            alt.Tooltip("directorate:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("cancelled_count:Q", title="Cancelled grants"),
            alt.Tooltip(
                "cancelled_amount:Q", title="Cancelled amount ($)", format=",.0f"
            ),
        ],
    )
    .transform_filter(dir_sel)
    .properties(
        width=750, height=180, title="Selected directorate — cancellations over time"
    )
)

(q3_scatter & q3_bars & q3_trend)

alt.VConcatChart(...)

In [103]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. DATA PREPARATION
# A. Cancelled Stats (Trump Era)
q3_cancel_df = (
    df_trump.groupby(["directorate"])
    .agg(
        cancelled_count=("award_id", "count"), cancelled_amount=("award_amount", "sum")
    )
    .reset_index()
)

# B. Base Stats (Total Grants)
q3_base_df = (
    df_grants.groupby(["directorate"])
    .agg(base_count=("award_id", "count"), base_amount=("award_amount", "sum"))
    .reset_index()
)

# C. Merge & Calculate Rate
q3_df = q3_base_df.merge(q3_cancel_df, on="directorate", how="outer").fillna(0)
q3_df["cancel_rate"] = (
    q3_df["cancelled_count"] / q3_df["base_count"].replace(0, 1)
).fillna(0)

# Filter noise (must have at least 1 grant to be relevant)
q3_scatter_df = q3_df[(q3_df["base_count"] > 0)].copy()

# D. Trend Data (for the line chart)
q3_trend_data = (
    df_trump.groupby(["directorate", "year"])
    .agg(cancelled_count=("award_id", "count"))
    .reset_index()
)

# 2. CREATE SELECTION (Robust for older Altair)
try:
    dir_select = alt.selection_point(fields=["directorate"], empty="all")
except AttributeError:
    dir_select = alt.selection_single(fields=["directorate"], empty="all")

# 3. LEFT CHART: THE RADAR (Scatter)
# X = Volume (Log scale handles the huge range)
# Y = Intensity (Rate)
scatter = (
    alt.Chart(q3_scatter_df)
    .mark_circle(size=100, stroke="black", strokeWidth=0.5, opacity=0.8)
    .encode(
        x=alt.X(
            "base_count:Q",
            scale=alt.Scale(type="log"),
            title="Total Grants (Log Scale)",
        ),
        y=alt.Y("cancel_rate:Q", title="Cancellation Rate", axis=alt.Axis(format="%")),
        color=alt.condition(
            dir_select,
            alt.Color("cancel_rate:Q", scale=alt.Scale(scheme="reds"), legend=None),
            alt.value("#f0f0f0"),  # Turn gray if not clicked
        ),
        size=alt.Size("cancelled_amount:Q", title="Lost Funding ($)", legend=None),
        tooltip=[
            alt.Tooltip("directorate:N", title="Directorate"),
            alt.Tooltip("base_count:Q", title="Total Grants"),
            alt.Tooltip("cancel_rate:Q", title="Cancel Rate", format=".1%"),
            alt.Tooltip("cancelled_amount:Q", title="Lost Funding", format="$,.0f"),
        ],
    )
    .add_selection(dir_select)  # <--- Interaction Driver
    .properties(
        width=450, height=400, title="Q3: Cancellation Intensity (Rate vs Volume)"
    )
)

# Add a mean line for context
mean_rate = q3_scatter_df["cancel_rate"].mean()
rule = (
    alt.Chart(pd.DataFrame({"mean_rate": [mean_rate]}))
    .mark_rule(color="gray", strokeDash=[4, 4])
    .encode(y="mean_rate:Q")
)

left_chart = scatter + rule

# 4. RIGHT COLUMN COMPONENTS

# A. Spacer (Top margin)
top_spacer = (
    alt.Chart(q3_df)
    .mark_rect()
    .encode(opacity=alt.value(0))
    .properties(width=300, height=50)
)

# B. Trend Chart (Red)
trend_chart = (
    alt.Chart(q3_trend_data)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("cancelled_count:Q", title="Cancellations"),
        color=alt.value("#d62728"),  # Standard Red
        tooltip=["directorate", "year", "cancelled_count"],
    )
    .transform_filter(dir_select)
    .properties(width=300, height=200, title="Cancellation History (Selected)")
)

# C. KPI Text
base_text = alt.Chart(q3_df).transform_filter(dir_select)

label = base_text.mark_text(align="center", color="#888", fontSize=14, dy=-15).encode(
    text=alt.value("Total Lost Funding"), y=alt.value(60), x=alt.value(150)
)

value = base_text.mark_text(
    align="center", color="#333", fontSize=24, fontWeight="bold", dy=15
).encode(
    text=alt.Text("sum(cancelled_amount):Q", format="$,.0f"),
    y=alt.value(60),
    x=alt.value(150),
)

kpi_section = (label + value).properties(width=300, height=120)

# D. Legend (Dummy Chart)
legend_chart = (
    alt.Chart(q3_df)
    .mark_circle(opacity=0)
    .encode(
        color=alt.Color(
            "cancel_rate:Q",
            scale=alt.Scale(scheme="reds"),
            legend=alt.Legend(
                title="Cancellation Intensity (Rate)",
                orient="bottom",
                direction="horizontal",
                titleAnchor="middle",
                gradientLength=200,
            ),
        )
    )
    .properties(width=300, height=50)
)

# 5. ASSEMBLE
right_col = alt.vconcat(top_spacer, trend_chart, kpi_section, legend_chart, spacing=10)

final_q3 = (
    (left_chart | right_col)
    .resolve_scale(color="independent")
    .configure_view(stroke=None)
    .configure_concat(spacing=30)
)

final_q3

/var/folders/xz/0gtb2skn4gl_q4q7wvvth0g40000gn/T/ipykernel_88995/1116593365.py:72: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(dir_select)  # <--- Interaction Driver


alt.HConcatChart(...)

comments

## Q4: How have the total grants amount evolved over the years?

In [102]:
import pandas as pd
import altair as alt

# --------------------------------------------------
# DATA AGGREGATION
# --------------------------------------------------

q4_df = (
    df_grants.groupby("year")
    .agg(total_amount=("award_amount", "sum"), grants_count=("award_id", "count"))
    .reset_index()
    .sort_values("year")
)

# --------------------------------------------------
# INTERACTION: YEAR SELECTION
# --------------------------------------------------

year_sel = alt.selection_point(fields=["year"], empty="all")

# --------------------------------------------------
# MAIN TREND: TOTAL FUNDING (PRIMARY STORY)
# --------------------------------------------------

q4_funding = (
    alt.Chart(q4_df)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y(
            "total_amount:Q", title="Total NSF funding ($)", axis=alt.Axis(format="~s")
        ),
        color=alt.condition(year_sel, alt.value("#1f77b4"), alt.value("#b0c4de")),
        tooltip=[
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
            alt.Tooltip("grants_count:Q", title="Number of grants"),
        ],
    )
    .add_params(year_sel)
    .properties(
        width=750,
        height=300,
        title="Q4 — Evolution of total NSF funding over the last 5 years",
    )
)

# --------------------------------------------------
# CONTEXT: NUMBER OF GRANTS (SECONDARY STORY)
# --------------------------------------------------

q4_grants = (
    alt.Chart(q4_df)
    .mark_bar()
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("grants_count:Q", title="Number of grants"),
        color=alt.condition(year_sel, alt.value("#ff7f0e"), alt.value("#ffd8b1")),
        tooltip=[
            alt.Tooltip("year:O"),
            alt.Tooltip("grants_count:Q", title="Number of grants"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
        ],
    )
    .add_params(year_sel)
    .properties(width=750, height=200, title="Number of grants per year (context)")
)

# --------------------------------------------------
# FINAL COMPOSITION
# --------------------------------------------------

(q4_funding & q4_grants)

alt.VConcatChart(...)

needs work, very bad

## Q5: For a selected state, how have the grants evolved? Are there cancelled grants?

In [105]:
import altair as alt
import pandas as pd

# 0. SETUP
alt.data_transformers.enable("default")

# 1. LOAD DATA
# Ensure you have these files in your folder
df_grants = pd.read_csv("NSF_Grants_Last5Years_Clean.csv")
df_trump = pd.read_csv("trump17-21-csv.csv")

# Clean columns just in case
df_grants.columns = df_grants.columns.str.strip()
df_trump.columns = df_trump.columns.str.strip()

# 2. AGGREGATIONS
q5_grants = (
    df_grants.groupby(["state", "year"])
    .agg(grants_count=("award_id", "count"), total_amount=("award_amount", "sum"))
    .reset_index()
)

q5_trump = (
    df_trump.groupby(["state", "year"])
    .agg(
        cancelled_count=("award_id", "count"), cancelled_amount=("award_amount", "sum")
    )
    .reset_index()
)

# 3. CREATE SELECTION (Dropdown)
# Get list of states for the dropdown
states = sorted(q5_grants["state"].unique())
state_dropdown = alt.binding_select(options=states, name="Select State: ")

try:
    # Modern Altair
    state_selection = alt.selection_point(
        fields=["state"], bind=state_dropdown, value=[{"state": "CA"}]
    )
except AttributeError:
    # Older Altair
    state_selection = alt.selection_single(
        fields=["state"], bind=state_dropdown, init={"state": "CA"}
    )

# 4. CHART DEFINITIONS

# Chart A: Total Funding (Line)
q5_amount_line = (
    alt.Chart(q5_grants)
    .mark_line(point=True, color="#4c78a8")  # Standard Blue
    .encode(
        x=alt.X("year:O", title="Year (last 5 years)"),
        y=alt.Y(
            "total_amount:Q", title="Total Funding ($)", axis=alt.Axis(format="~s")
        ),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
            alt.Tooltip("grants_count:Q", title="Number of grants"),
        ],
    )
    .add_selection(state_selection)  # <--- Selection added here
    .transform_filter(state_selection)
    .properties(
        width=750, height=200, title="Q5 — Selected State: Total Funding Evolution"
    )
)

# Chart B: Number of Grants (Bar)
q5_count_bar = (
    alt.Chart(q5_grants)
    .mark_bar(color="#72b7b2")  # Teal
    .encode(
        x=alt.X("year:O", title="Year (last 5 years)"),
        y=alt.Y("grants_count:Q", title="Number of grants"),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("grants_count:Q", title="Number of grants"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
        ],
    )
    .transform_filter(state_selection)  # Listens to the same selection
    .properties(width=750, height=150, title="Grant Count Evolution")
)

# Chart C: Cancelled Grants (Bar - Red)
q5_cancelled = (
    alt.Chart(q5_trump)
    .mark_bar(color="#e45756")  # Red
    .encode(
        x=alt.X("year:O", title="Year (Trump era)"),
        y=alt.Y("cancelled_count:Q", title="Cancelled grants"),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("cancelled_count:Q", title="Cancelled grants"),
            alt.Tooltip(
                "cancelled_amount:Q", title="Cancelled amount ($)", format=",.0f"
            ),
        ],
    )
    .transform_filter(state_selection)  # Listens to the same selection
    .properties(width=750, height=150, title="Trump Era (2017–2021): Cancelled Grants")
)

# 5. ASSEMBLE
final_q5 = q5_amount_line & q5_count_bar & q5_cancelled

final_q5

/var/folders/xz/0gtb2skn4gl_q4q7wvvth0g40000gn/T/ipykernel_88995/1516382373.py:65: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(state_selection)  # <--- Selection added here


alt.VConcatChart(...)

In [ ]:
df_grants.columns = df_grants.columns.str.strip()
df_trump.columns = df_trump.columns.str.strip()


In [ ]:
q5_grants = (
    df_grants
    .groupby(["state", "year"])
    .agg(
        grants_count=("award_id", "count"),
        total_amount=("award_amount", "sum")
    )
    .reset_index()
)


In [ ]:
q5_trump = (
    df_trump
    .groupby(["state", "year"])
    .agg(
        cancelled_count=("award_id", "count"),
        cancelled_amount=("award_amount", "sum")
    )
    .reset_index()
)


In [ ]:
q5_amount_line = (
    alt.Chart(q5_grants)
    .transform_filter(state_selection)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year (last 5 years)"),
        y=alt.Y("total_amount:Q", title="Total funding ($)", axis=alt.Axis(format="~s")),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
            alt.Tooltip("grants_count:Q", title="Number of grants")
        ]
    )
    .add_params(state_selection)
    .properties(width=750, height=260, title="Q5 — Selected state: total funding over time (2020–2024)")
)


In [ ]:
q5_count_bar = (
    alt.Chart(q5_grants)
    .transform_filter(state_selection)
    .mark_bar()
    .encode(
        x=alt.X("year:O", title="Year (last 5 years)"),
        y=alt.Y("grants_count:Q", title="Number of grants"),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("grants_count:Q", title="Number of grants"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
        ]
    )
    .add_params(state_selection)
    .properties(width=750, height=200, title="Selected state: number of grants per year (2020–2024)")
)


In [ ]:
q5_cancelled = (
    alt.Chart(q5_trump)
    .transform_filter(state_selection)
    .mark_bar()
    .encode(
        x=alt.X("year:O", title="Year (Trump era)"),
        y=alt.Y("cancelled_count:Q", title="Cancelled grants"),
        tooltip=[
            alt.Tooltip("state:N"),
            alt.Tooltip("year:O"),
            alt.Tooltip("cancelled_count:Q", title="Cancelled grants"),
            alt.Tooltip("cancelled_amount:Q", title="Cancelled amount ($)", format=",.0f")
        ]
    )
    .add_params(state_selection)
    .properties(width=750, height=200, title="Trump era (2017–2021): cancelled grants for selected state")
)


In [ ]:
(q5_amount_line & q5_count_bar & q5_cancelled)


alt.VConcatChart(...)

comments

## Q6: Select some attribute that has not been mentioned previously (e.g., party governing, population of the state, number of funded institutions in the state…), and let the user interactively explore the information around the attribute to get insights.

For Question 6, state population was selected as an additional attribute not previously used in the analysis. Population is a meaningful contextual variable that enables deeper exploration beyond absolute grant counts or total funding amounts. By relating funding to population size, users can investigate whether certain states receive disproportionately high or low levels of funding relative to their population, revealing patterns that are not visible through raw totals alone.

This attribute supports an analysis by enabling per capita comparisons, outlier detection, and interactive investigation of funding efficiency across states and years. It integrates naturally with the existing state based aggregations used in earlier questions.

In [54]:
import pandas as pd
import altair as alt

df_pop_raw = pd.read_csv("estimated_population.csv")
df_abbr_raw = pd.read_csv("state_abbreviations.csv")

# Clean column names
df_pop_raw.columns = df_pop_raw.columns.str.strip()
df_abbr_raw.columns = df_abbr_raw.columns.str.strip()

In [55]:
# Ensure we have a 'state' column (full names like Alabama, Alaska, ...)
if "state" not in df_pop_raw.columns:
    raise ValueError(f"estimated_population.csv must have a 'state' column. Found: {list(df_pop_raw.columns)}")

pop_cols = [c for c in df_pop_raw.columns if c.lower().startswith("pop_")]
if not pop_cols:
    raise ValueError(f"Could not find pop_YYYY columns. Found: {list(df_pop_raw.columns)}")

df_pop_long = df_pop_raw.melt(
    id_vars=["state"],
    value_vars=pop_cols,
    var_name="year",
    value_name="population"
)

# Convert year from 'pop_2020' -> 2020
df_pop_long["year"] = df_pop_long["year"].str.replace("pop_", "", regex=False).astype(int)

# Convert population to numeric
df_pop_long["population"] = pd.to_numeric(df_pop_long["population"], errors="coerce")

# Keep only 2020-2024 (safety)
df_pop_long = df_pop_long[df_pop_long["year"].between(2020, 2024)]

# Standardize state name
df_pop_long = df_pop_long.rename(columns={"state": "state_name"})
df_pop_long["state_name"] = df_pop_long["state_name"].astype(str).str.strip()

In [56]:
df_abbr = df_abbr_raw.copy()

# Detect likely columns for state name and abbreviation
name_candidates = [c for c in df_abbr.columns if "name" in c.lower() or ("state" in c.lower() and "abbr" not in c.lower())]
abbr_candidates = [c for c in df_abbr.columns if "abbr" in c.lower() or "code" in c.lower()]

if not name_candidates or not abbr_candidates:
    raise ValueError(
        "state_abbreviations.csv must contain columns for full state name and abbreviation.\n"
        f"Columns found: {list(df_abbr.columns)}"
    )

name_col = name_candidates[0]
abbr_col = abbr_candidates[0]

df_abbr = df_abbr.rename(columns={name_col: "state_name", abbr_col: "state"})
df_abbr["state_name"] = df_abbr["state_name"].astype(str).str.strip()
df_abbr["state"] = df_abbr["state"].astype(str).str.strip()

# Normalize case (helps joins)
df_abbr["state_name_key"] = df_abbr["state_name"].str.lower()
df_pop_long["state_name_key"] = df_pop_long["state_name"].str.lower()

# Join to add 2-letter codes
df_pop_long = df_pop_long.merge(
    df_abbr[["state_name_key", "state"]],
    on="state_name_key",
    how="left"
)

# Debug unmapped names
unmapped = df_pop_long[df_pop_long["state"].isna()]["state_name"].dropna().unique()
print("Unmapped population state names (should be empty):", unmapped[:20], " ... total:", len(unmapped))

# Keep only mapped rows + required cols
df_pop_long = df_pop_long.dropna(subset=["state", "population"])
df_pop_long = df_pop_long[["state", "year", "population"]].copy()

print("Population long shape:", df_pop_long.shape)
print("Population states:", df_pop_long["state"].nunique(), "Years:", sorted(df_pop_long["year"].unique()))

Unmapped population state names (should be empty): []  ... total: 0
Population long shape: (255, 3)
Population states: 51 Years: [2020, 2021, 2022, 2023, 2024]


In [57]:
# sanity: ensure expected columns exist
required_cols = {"state", "year", "award_amount", "award_id"}
missing = required_cols - set(df_grants.columns)
if missing:
    raise ValueError(f"df_grants missing required columns: {missing}. Found: {list(df_grants.columns)}")

# Ensure year numeric
df_grants["year"] = pd.to_numeric(df_grants["year"], errors="coerce").astype("Int64")

q6_grants = (
    df_grants
    .dropna(subset=["state", "year", "award_amount"])
    .groupby(["state", "year"])
    .agg(
        total_amount=("award_amount", "sum"),
        grants_count=("award_id", "count")
    )
    .reset_index()
)

print("NSF aggregated shape:", q6_grants.shape)
print("NSF states:", q6_grants["state"].nunique(), "Years:", sorted(q6_grants["year"].unique()))


NSF aggregated shape: (231, 4)
NSF states: 52 Years: [2020, 2021, 2022, 2023, 2024]


In [58]:
q6_df = q6_grants.merge(df_pop_long, on=["state", "year"], how="inner")
q6_df["funding_per_capita"] = q6_df["total_amount"] / q6_df["population"]

print("Merged q6_df shape:", q6_df.shape)
print("Merged states:", q6_df["state"].nunique(), "Years:", sorted(q6_df["year"].unique()))

# If still empty, show mismatch hints
if q6_df.empty:
    print("\nq6_df is EMPTY. Debug hints:")
    print("Sample NSF states:", sorted(q6_grants["state"].unique())[:15])
    print("Sample POP states:", sorted(df_pop_long["state"].unique())[:15])
    print("Sample NSF years:", sorted(q6_grants["year"].unique()))
    print("Sample POP years:", sorted(df_pop_long["year"].unique()))
    raise ValueError("Merge produced empty q6_df. See debug hints above.")

Merged q6_df shape: (222, 6)
Merged states: 50 Years: [2020, 2021, 2022, 2023, 2024]


In [59]:
year_selection = alt.selection_point(
    fields=["year"],
    bind=alt.binding_select(options=sorted(q6_df["year"].unique()), name="Year: "),
    value=max(q6_df["year"].unique())
)

state_click = alt.selection_point(fields=["state"], empty="all")

q6_overview = (
    alt.Chart(q6_df)
    .mark_bar()
    .encode(
        x=alt.X("state:N", sort="-y", title="State"),
        y=alt.Y("funding_per_capita:Q", title="Funding per capita ($)", axis=alt.Axis(format="~s")),
        color=alt.condition(
            state_click,
            alt.Color("funding_per_capita:Q", scale=alt.Scale(scheme="purples"), title="Funding per capita"),
            alt.value("lightgray")
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("population:Q", title="Population", format=",.0f"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
            alt.Tooltip("funding_per_capita:Q", title="Funding per capita ($)", format=",.2f"),
            alt.Tooltip("grants_count:Q", title="Grants count"),
        ]
    )
    .add_params(year_selection, state_click)
    .transform_filter(year_selection)
    .properties(width=750, height=380, title="Q6 — NSF funding per capita by state (select year + click a state)")
)

In [60]:

q6_trend = (
    alt.Chart(q6_df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("funding_per_capita:Q", title="Funding per capita ($)", axis=alt.Axis(format="~s")),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("year:O", title="Year"),
            alt.Tooltip("funding_per_capita:Q", title="Funding per capita ($)", format=",.2f"),
            alt.Tooltip("total_amount:Q", title="Total funding ($)", format=",.0f"),
            alt.Tooltip("population:Q", title="Population", format=",.0f"),
            alt.Tooltip("grants_count:Q", title="Grants count"),
        ]
    )
    .transform_filter(state_click)
    .properties(width=750, height=200, title="Selected state — funding per capita over time (2020–2024)")
)

(q6_overview & q6_trend)


alt.VConcatChart(...)

comment

# Final Visualization